# Ollama Setup and Testing

This notebook will help you set up and test Ollama with LangChain connectors before starting the main RAG assignment.

## Prerequisites

1. **Install Ollama** from https://ollama.ai
   - On Linux/Mac: `curl https://ollama.ai/install.sh | sh`
   - On Windows: Download and run the installer

2. **Verify Installation**
   - Run `ollama -v` in your terminal
   - Should show version 0.11.10 or greater

3. **Pull Required Models**
   ```bash
   # For chat/inference
   ollama pull gpt-oss:20b
   
   # For embeddings
   ollama pull embeddinggemma:latest
   ```


## Step 1: Test Ollama Connection

First, let's verify that Ollama is running and accessible:


In [1]:
import requests
import json

# Test if Ollama is running
try:
    response = requests.get('http://localhost:11434/api/tags')
    if response.status_code == 200:
        models = json.loads(response.text)
        print("✅ Ollama is running!")
        print("\nAvailable models:")
        for model in models.get('models', []):
            print(f"  - {model['name']}")
    else:
        print("❌ Ollama is not responding properly")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to Ollama. Make sure it's running!")
    print("Start Ollama by running 'ollama serve' in a terminal")


✅ Ollama is running!

Available models:
  - embeddinggemma:latest
  - gpt-oss:20b
  - deepseek-r1:latest
  - mxbai-embed-large:latest
  - deepseek-r1:8b


## Step 2: Test Embeddings with Ollama

Now let's test creating embeddings using the LangChain Ollama connector:


In [ ]:
from langchain_ollama import OllamaEmbeddings

# Initialize the embedding model
embedding_model = OllamaEmbeddings(
    model="embeddinggemma:latest",
    base_url="http://localhost:11434"  # Default Ollama URL
)

print("✅ Embedding model initialized")

✅ Embedding model initialized


In [3]:
# Test embedding a single query
test_query = "What is the meaning of life?"

print(f"Embedding query: '{test_query}'")
embedding = embedding_model.embed_query(test_query)

print(f"\n✅ Successfully created embedding!")
print(f"Embedding dimension: {len(embedding)}")
print(f"First 10 values: {embedding[:10]}")


Embedding query: 'What is the meaning of life?'

✅ Successfully created embedding!
Embedding dimension: 768
First 10 values: [-0.14612523, 0.029183429, 0.037420813, -0.02506321, -0.026125213, 0.016262747, -0.026964674, 0.026944311, 0.0111782, -4.2271626e-05]


In [4]:
# Test embedding multiple documents
test_documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is a subset of artificial intelligence.",
    "Python is a popular programming language for data science."
]

print("Embedding multiple documents...")
embeddings = embedding_model.embed_documents(test_documents)

print(f"\n✅ Successfully created {len(embeddings)} embeddings!")
for i, doc in enumerate(test_documents):
    print(f"\nDocument {i+1}: '{doc[:50]}...'")
    print(f"  Embedding dimension: {len(embeddings[i])}")
    print(f"  First 5 values: {embeddings[i][:5]}")


Embedding multiple documents...

✅ Successfully created 3 embeddings!

Document 1: 'The quick brown fox jumps over the lazy dog....'
  Embedding dimension: 768
  First 5 values: [-0.14769432, 0.0027910522, 0.05211773, -0.028957522, -0.03694144]

Document 2: 'Machine learning is a subset of artificial intelli...'
  Embedding dimension: 768
  First 5 values: [-0.12411143, -0.0027318636, -0.00045949276, 0.009990234, 0.0016487375]

Document 3: 'Python is a popular programming language for data ...'
  Embedding dimension: 768
  First 5 values: [-0.16281867, -0.010769529, 0.025246046, 0.0003698215, -0.018286267]


## Step 3: Test Model Inference with Ollama

Now let's test using Ollama for text generation/inference using the LangChain connector:


In [5]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize the chat model
chat_model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0.7,
    base_url="http://localhost:11434"
)

print("✅ Chat model initialized")


✅ Chat model initialized


In [6]:
# Test simple inference
prompt = "Explain quantum computing in one sentence."

print(f"Prompt: {prompt}")
print("\nGenerating response...")

response = chat_model.invoke(prompt)

print(f"\n✅ Response generated!")
print(f"\nModel output: {response.content}")


Prompt: Explain quantum computing in one sentence.

Generating response...

✅ Response generated!

Model output: Quantum computing harnesses quantum bits that can exist in multiple states at once, enabling it to solve certain problems exponentially faster than classical computers.


In [7]:
# Test with system message and human message
messages = [
    SystemMessage(content="You are a helpful AI assistant that explains complex topics simply."),
    HumanMessage(content="What is machine learning?")
]

print("Sending messages to model...")
response = chat_model.invoke(messages)

print(f"\n✅ Response generated!")
print(f"\nModel output: {response.content}")


Sending messages to model...

✅ Response generated!

Model output: **Machine learning is a way of teaching computers to learn from experience, just like people do.**

---

### 1. The big idea
- **Instead of writing every rule by hand**, we give a computer a lot of data and let it discover the patterns itself.
- The computer builds a *model* (a mathematical recipe) that can then make predictions or decisions on new, unseen data.

### 2. How it works in plain terms

| Step | What happens | Simple example |
|------|--------------|----------------|
| **Collect data** | Gather lots of examples. | Photos of cats and dogs. |
| **Choose a task** | Decide what you want the model to do. | Tell the computer “label each photo as cat or dog.” |
| **Teach the model** | Let the computer look at the data and adjust its internal settings to fit the examples. | The computer adjusts weights in a neural network so that it correctly identifies cats. |
| **Test it** | Give it new data it hasn’t seen before 

## Step 4: Test Streaming Response

Ollama supports streaming responses, which is useful for real-time applications:


In [10]:
# Test streaming
prompt = "Write a haiku about artificial intelligence."

print(f"Prompt: {prompt}")
print("\nStreaming response:")
print("-" * 40)

for chunk in chat_model.stream(prompt):
    print(chunk.content, end="", flush=True)

print("\n" + "-" * 40)
print("\n✅ Streaming completed!")


Prompt: Write a haiku about artificial intelligence.

Streaming response:
----------------------------------------
Silent code breathes deep  
Neural nets hum quietly  
Future breathes in code
----------------------------------------

✅ Streaming completed!


## Summary

If all the tests above passed, you're ready to use Ollama with LangChain! Here's what we tested:

✅ **Embeddings**: 
- Created embeddings for single queries
- Created embeddings for multiple documents
- Verified embedding dimensions

✅ **Model Inference**:
- Simple text generation
- Chat with system and human messages
- Streaming responses
- Integration with LangChain chains

## Troubleshooting

If you encounter issues:

1. **Model Not Found**: Pull the required models (`ollama pull <model-name>`)
2. **Slow Performance**: Ollama models run on CPU by default. For better performance:
   - Use smaller models for testing
   - Consider GPU acceleration if available
3. **Memory Issues**: Large models require significant RAM. Try smaller variants if needed.

## Next Steps

Now you're ready to proceed with the main RAG assignment using Ollama!
